In [1]:
# Instale as dependências necessárias
!pip install pyspark findspark

In [2]:
import wget
wget.download("https://jdbc.postgresql.org/download/postgresql-42.7.0.jar")

'postgresql-42.7.0 (1).jar'

In [3]:
from pyspark.sql import SparkSession

# Configuração do Spark
spark = SparkSession.builder \
    .appName("Nome do Aplicativo") \
    .config("spark.jars", "./postgresql-42.7.0.jar") \
    .config("spark.driver.extraClassPath", "./postgresql-42.7.0.jar") \
    .master("spark://spark-master:7077")\
    .config("spark.executor.memory", "512m")\
    .getOrCreate()

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
23/12/03 13:32:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
# Conectar ao banco de dados
jdbc_url = "jdbc:postgresql://172.22.0.2:5432/teste_db"
properties = {
    "user": "root",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

df = spark.read.jdbc(jdbc_url, "livros", properties=properties)

In [5]:
# Realizar a leitura em chunks do arquivo CSV
data_chunks = spark.read.option("encoding", "ISO-8859-1") \
    .option("sep", ";") \
    .option("header", False) \
    .option("inferSchema", False) \
    .csv("/opt/workspace/datain/estabelecimentos/*.csv")

In [7]:
data_chunks.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 '_c8',
 '_c9',
 '_c10',
 '_c11',
 '_c12',
 '_c13',
 '_c14',
 '_c15',
 '_c16',
 '_c17',
 '_c18',
 '_c19',
 '_c20',
 '_c21',
 '_c22',
 '_c23',
 '_c24',
 '_c25',
 '_c26',
 '_c27',
 '_c28',
 '_c29']

In [8]:
# rename the columns
old_columns = data_chunks.columns
new_columns = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv','id_matriz_filial','nome_fantasia','situacao_cadastral','data_situacao_cadastral',
'motivo_situacao_cadastral','nome_exterior','pais','data_inicio_atividade','cnae_pri','cnae_sec','tipo_logradouro','logradouro','numero','complemento','bairro','cep','uf','municipio','dd1','telefone1','dd2','telefone2','ddd_fax','fax','email','situacao_especial','data_situacao_especial']

def renameCols(df, old_columns, new_columns):
    for old_col,new_col in zip(old_columns,new_columns):
        df = df.withColumnRenamed(old_col,new_col)
    return df

df = renameCols(data_chunks, old_columns, new_columns)

In [9]:
df.columns

['cnpj_basico',
 'cnpj_ordem',
 'cnpj_dv',
 'id_matriz_filial',
 'nome_fantasia',
 'situacao_cadastral',
 'data_situacao_cadastral',
 'motivo_situacao_cadastral',
 'nome_exterior',
 'pais',
 'data_inicio_atividade',
 'cnae_pri',
 'cnae_sec',
 'tipo_logradouro',
 'logradouro',
 'numero',
 'complemento',
 'bairro',
 'cep',
 'uf',
 'municipio',
 'dd1',
 'telefone1',
 'dd2',
 'telefone2',
 'ddd_fax',
 'fax',
 'email',
 'situacao_especial',
 'data_situacao_especial']

In [10]:
df.show(5)

23/12/03 13:36:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+----------+-------+----------------+-----------------+------------------+-----------------------+-------------------------+-------------+----+---------------------+--------+--------------------+---------------+--------------------+------+--------------+---------------+--------+---+---------+---+---------+----+---------+-------+--------+--------------------+-----------------+----------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|id_matriz_filial|    nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_exterior|pais|data_inicio_atividade|cnae_pri|            cnae_sec|tipo_logradouro|          logradouro|numero|   complemento|         bairro|     cep| uf|municipio|dd1|telefone1| dd2|telefone2|ddd_fax|     fax|               email|situacao_especial|data_situacao_especial|
+-----------+----------+-------+----------------+-----------------+------------------+-----------------------+-------------------------+-------------+----+-----------------

In [11]:
from pyspark.sql import functions as F
data_filtered = df.filter((F.col("cnae_pri") == 8121400) & (F.col("uf") == 'RN'))

In [12]:
data_filtered.count()

51

In [13]:
data_filtered.show()

+-----------+----------+-------+----------------+--------------------+------------------+-----------------------+-------------------------+-------------+----+---------------------+--------+--------------------+---------------+--------------------+------+--------------------+-------------------+--------+---+---------+----+---------+----+---------+-------+----+--------------------+-----------------+----------------------+
|cnpj_basico|cnpj_ordem|cnpj_dv|id_matriz_filial|       nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_exterior|pais|data_inicio_atividade|cnae_pri|            cnae_sec|tipo_logradouro|          logradouro|numero|         complemento|             bairro|     cep| uf|municipio| dd1|telefone1| dd2|telefone2|ddd_fax| fax|               email|situacao_especial|data_situacao_especial|
+-----------+----------+-------+----------------+--------------------+------------------+-----------------------+-------------------------+-------------

In [14]:
data_chunks1 = spark.read.option("encoding", "ISO-8859-1") \
    .option("sep", ";") \
    .option("header", False) \
    .option("inferSchema", False) \
    .csv("./datain/empresas.csv")

In [16]:
data_chunks1.columns

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6']

In [17]:
# rename the columns
old_columns = data_chunks1.columns
new_columns = ['cnpj_basico', 'razao_social','natureza_juridica','qualificacao','capital_social', 'porte_empresa','responsavel']

def renameCols1(df1, old_columns1, new_columns1):
    for old_col,new_col in zip(old_columns1,new_columns1):
        df1 = df1.withColumnRenamed(old_col,new_col)
    return df1

df1 = renameCols1(data_chunks1, old_columns, new_columns)

In [18]:
df1.show()

+-----------+--------------------+-----------------+------------+--------------+-------------+-----------+
|cnpj_basico|        razao_social|natureza_juridica|qualificacao|capital_social|porte_empresa|responsavel|
+-----------+--------------------+-----------------+------------+--------------+-------------+-----------+
|   41273596|RODRIGO JOSE FERR...|             2135|          50|      10000,00|           01|       null|
|   41273597|PACHARRUS QUEIROZ...|             2135|          50|       5000,00|           01|       null|
|   41273598|GLORIA VIANA DIAS...|             2135|          50|       1100,00|           01|       null|
|   41273599|ANA PAULA DA SILV...|             2135|          50|       2000,00|           01|       null|
|   41273600|41.273.600 AVANIL...|             2135|          50|      50000,00|           01|       null|
|   41273601|GABRIELA HELENA F...|             2135|          50|       2000,00|           01|       null|
|   41273602|FABIO SOUZA DO RO...|   

In [19]:
# Realizar uma junção (join) com o DataFrame df1 usando a coluna comum 'cnpj_basico'
result_df = data_filtered.join(df1, on='cnpj_basico', how='left_outer')

In [20]:
result_df.show()

+-----------+----------+-------+----------------+--------------------+------------------+-----------------------+-------------------------+-------------+----+---------------------+--------+--------------------+---------------+--------------------+------+-----------------+--------------+--------+---+---------+----+---------+----+---------+-------+--------+--------------------+-----------------+----------------------+------------+-----------------+------------+--------------+-------------+-----------+
|cnpj_basico|cnpj_ordem|cnpj_dv|id_matriz_filial|       nome_fantasia|situacao_cadastral|data_situacao_cadastral|motivo_situacao_cadastral|nome_exterior|pais|data_inicio_atividade|cnae_pri|            cnae_sec|tipo_logradouro|          logradouro|numero|      complemento|        bairro|     cep| uf|municipio| dd1|telefone1| dd2|telefone2|ddd_fax|     fax|               email|situacao_especial|data_situacao_especial|razao_social|natureza_juridica|qualificacao|capital_social|porte_empresa|res

In [26]:
# Reduzir o número de partições para 1 antes de salvar
data_filtered_single_partition = result_df.coalesce(1)

In [27]:
data_filtered_single_partition.write.format("csv") \
    .mode("overwrite") \
    .option("header", True) \
    .save("/opt/workspace/dataout/")

In [23]:
result_df.write.format('jdbc').options(
      url='jdbc:postgresql://172.22.0.2:5432/teste_db',
      driver='org.postgresql.Driver',
      dbtable='empresas',
      user='root',
      password='root').mode('append').save()

In [ ]:
spark.stop()